## Imports

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.6 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import T5ForConditionalGeneration
import numpy as np

In [3]:
# set device
device = torch.device("cuda")

In [4]:
# load t5
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")

t5_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
# load gpt2
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")

gpt_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

## T5 Work (reproducing/tweaking [existing CKA method](https://github.com/dqxiu/CaliNet/blob/master/cka/assessing_score.py))

In [77]:
def probe_t5(model,input_ids, target):
    # T5 is an encoder-decoder mode, so we have to provide both input_ids and decoder_input_ids to the model
    # the input id's correspond to the 
    # where 0 is the corresponding id for <pad>
    # and 32099 is the corresponding id for <extra_id_0>
    outputs = model(input_ids=input_ids, decoder_input_ids=torch.tensor([[0, 32099]],device='cuda:0'),
                    output_hidden_states=True, return_dict=True)
    
    # outputs contains:
      # logits -- Prediction scores of the language modeling head)
        # unnormalized scores for each possible token at the masked token position
      # past_key_values -- Contains pre-computed hidden-states (key and values in the attention blocks) 
      # decoder_hidden_states  -- Hidden-states of the decoder at the output of each layer plus the initial embedding outputs.
      # encoder_last_hidden_state -- Sequence of hidden-states at the output of the last layer of the encoder of the model.
      # encoder_hidden_states -- Hidden-states of the encoder at the output of each layer plus the initial embedding outputs.
    
    # torch.Size([1, 2, 32128]) 
    # 1 x 2 x 32128 because 
    # we have batch_size of 1
    # and a sequence length of 2
    # and the vocab size for t5 is 32128

    # torch.Size([32128])
    # We have batch size of 1, so grab that, then, 
    # Take the entire last matrix which corresponds to the last layer
    logits = outputs['logits'][0, -1]

    # convert our prediction scores to a probability distribution with softmax
    # https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html#torch.nn.functional.softmax
    probs = F.softmax(logits, dim=-1)

    # originally, they had 
    # torch.reshape(probs, (-1,)).detach().cpu().numpy()
    # but that reshape line doesn't do anything since probs is already just 1-dimension (= to the embedding size)
    # in this case, for every token in T5's vocabulary, we're getting.. 
    # a prediction that the hidden token is that word?
    probs = probs.detach().cpu().numpy()

    # so let's just grab that by its given
    return probs[target.item()]

In [260]:
src_true = "Operation Overlord took place in <extra_id_0> ."
target_token = "France"

target = t5_tokenizer.encode(target_token, return_tensors="pt").to(device)[0][0]
            
input_ids = t5_tokenizer.encode(src_true, return_tensors="pt").to(device)

P_true = probe_t5(t5_model,input_ids, target) 

P_true

0.0018174488

In [261]:
src_true = "Operation Overlord took place in <extra_id_0> ."
target_token = "Belgium"

target = t5_tokenizer.encode(target_token, return_tensors="pt").to(device)[0][0]
            
input_ids = t5_tokenizer.encode(src_true, return_tensors="pt").to(device)

P_false = probe_t5(t5_model,input_ids, target) 

P_false

0.00039698844

In [262]:
P_true

0.0018174488

In [263]:
P_false

0.00039698844

In [264]:
P_true > P_false

True

## GPT2 work for CKA

In [245]:
def probe_gpt2(model, input_ids, target):

  # in case of multi tokenization .. --> is this tweak right?
  target_scalar = list(target.detach().cpu().numpy())

  # use model...
  outputs = gpt_model(input_ids=input_ids, output_hidden_states=True, return_dict=True)

  # shape of 50257 which corresponds to the vocab size of GPT
  logits = outputs['logits'][0, -1]

  probs = F.softmax(logits, dim=-1)

  probs = probs.detach().cpu().numpy()

  cumulative_prob = 0

  for t in target_scalar:
    cumulative_prob += probs[t]
    break

  return cumulative_prob

In [249]:
phrase = 'The 2020 Olympic Games took place in '
target_token = 'Tokyo'

target = gpt_tokenizer.encode(target_token, return_tensors='pt').to(device)[0]

input_ids = gpt_tokenizer.encode(phrase, return_tensors='pt').to(device)

P_true = probe_gpt2(t5_model,input_ids, target)

P_true

6.449251031881431e-06

In [248]:
phrase = 'The 2020 Olympic Games took place in '
target_token = 'Berlin'

target = gpt_tokenizer.encode(target_token, return_tensors='pt').to(device)[0]

input_ids = gpt_tokenizer.encode(phrase, return_tensors='pt').to(device)

P_false = probe_gpt2(t5_model,input_ids, target)

P_false

2.6307420739613008e-06

In [257]:
P_true > P_false

True